In [36]:
# Import packages 
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
import sklearn.metrics as metrics
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score
from sklearn import tree
from sklearn.model_selection import KFold, train_test_split
from sklearn.metrics import confusion_matrix
from sklearn import datasets
from sklearn.preprocessing import PolynomialFeatures
from time import sleep
import re
import requests
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

In [37]:
# Select URLs for scraping
url_10 = "https://en.wikipedia.org/wiki/2010_FIFA_World_Cup_squads"
url_14 = "https://en.wikipedia.org/wiki/2014_FIFA_World_Cup_squads"
url_18 = "https://en.wikipedia.org/wiki/2018_FIFA_World_Cup_squads"
url_22 = "https://en.wikipedia.org/wiki/2022_FIFA_World_Cup_squads"

page_10 = requests.get(url_10)
sleep(2)

page_14 = requests.get(url_14)
sleep(2)

page_18 = requests.get(url_18)
sleep(2)

page_22 = requests.get(url_22)
sleep(2)

page_10.status_code, page_14.status_code, page_18.status_code, page_22.status_code

(200, 200, 200, 200)

In [38]:
# Extract page HTML
raw_html_10 = page_10.text
raw_html_14 = page_14.text
raw_html_18 = page_18.text
raw_html_22 = page_22.text

page_soup_10 = BeautifulSoup(raw_html_10, 'html.parser')
page_soup_14 = BeautifulSoup(raw_html_14, 'html.parser')
page_soup_18 = BeautifulSoup(raw_html_18, 'html.parser')
page_soup_22 = BeautifulSoup(raw_html_22, 'html.parser')

In [39]:
# Define function to extract the list of countries
# participating in each world cup
def parse_country_list(page_soup):
    
    country_list = []
    country_elts = page_soup.select('h3')
    for tag in country_elts:
        modified_tag = tag.find('span', attrs = {'class': 'mw-headline'})
        if modified_tag is not None:  
            country_list.append(modified_tag.text)
            
    return country_list    

# Extract the participating country names for each tournament
countries_10 = parse_country_list(page_soup_10)
countries_10 = countries_10[:-5]

countries_14 = parse_country_list(page_soup_14)
countries_14 = countries_14[:-4]

countries_18 = parse_country_list(page_soup_18)
countries_18 = countries_18[:-5]

countries_22 = parse_country_list(page_soup_22)
countries_22 = countries_22[:-6]

print(countries_10)
print(countries_14)
print(countries_18)
print(countries_22)

['France', 'Mexico', 'South Africa', 'Uruguay', 'Argentina', 'Greece', 'Nigeria', 'South Korea', 'Algeria', 'England', 'Slovenia', 'United States', 'Australia', 'Germany', 'Ghana', 'Serbia', 'Cameroon', 'Denmark', 'Japan', 'Netherlands', 'Italy', 'New Zealand', 'Paraguay', 'Slovakia', 'Brazil', 'Ivory Coast', 'North Korea', 'Portugal', 'Chile', 'Honduras', 'Spain', 'Switzerland']
['Brazil', 'Cameroon', 'Croatia', 'Mexico', 'Australia', 'Chile', 'Netherlands', 'Spain', 'Colombia', 'Greece', 'Ivory Coast', 'Japan', 'Costa Rica', 'England', 'Italy', 'Uruguay', 'Ecuador', 'France', 'Honduras', 'Switzerland', 'Argentina', 'Bosnia and Herzegovina', 'Iran', 'Nigeria', 'Germany', 'Ghana', 'Portugal', 'United States', 'Algeria', 'Belgium', 'Russia', 'South Korea']
['Egypt', 'Russia', 'Saudi Arabia', 'Uruguay', 'Iran', 'Morocco', 'Portugal', 'Spain', 'Australia', 'Denmark', 'France', 'Peru', 'Argentina', 'Croatia', 'Iceland', 'Nigeria', 'Brazil', 'Costa Rica', 'Serbia', 'Switzerland', 'Germany',

In [40]:
import difflib

# Import player statisctics data and remove duplicate names
fifa_cleaned = pd.read_csv("fifa_cleaned.csv")
fifa_cleaned = fifa_cleaned.drop_duplicates(subset = 'full_name', keep = "first")

# Keep only the list of full names
fifa_names = list(fifa_cleaned.full_name)

# Create function which find squad and squad statistic data
def squad_parser(page_soup, countries_list):
    
    tables = page_soup.select('table.sortable.wikitable.plainrowheaders')
    players_list = []
    
    # Find the list of players for each country
    for i in range(len(countries_list)):
        rows = tables[i].select('.nat-fs-player')
        team_players = []
        
        for row in rows:
            
            player_dict = dict()
            
            # Scrape player position
            position = row.select_one('td a').text
            
            # Scrape player page for full name
            player_page = row.select_one('th a').get('href')
            player_page = "https://en.wikipedia.org" + player_page
            player_html = requests.get(player_page).text
            player_html = BeautifulSoup(player_html, 'html.parser')
            
            name = player_html.select_one('p b')
            full_name = name.text.strip()
            name_as_list = full_name.split()
            
            # Search for most similar name in FIFA video game name list
            if len(name_as_list) >= 2:
                short_name = name_as_list[0] + " " + name_as_list[-1]
                similar_name = difflib.get_close_matches(short_name, fifa_names, cutoff = 0.75)
                if len(similar_name) == 0:
                    similar_name = difflib.get_close_matches(full_name, fifa_names)
            else:
                similar_name = difflib.get_close_matches(full_name, fifa_names)
                
            if len(similar_name) >= 1:
                similar_name = similar_name[0]
            else:
                similar_name = None
            
            # Save player statistics
            player_dict['name'] = full_name
            player_dict['position'] = position
            player_dict['country'] = countries_list[i]
            
            # Find player ability data, if it exists
            if similar_name is None:
                player_dict['overall_rating'] = np.nan
                player_dict['dribbling'] = np.nan
                player_dict['freekick_accuracy'] = np.nan
                player_dict['agility'] = np.nan
                player_dict['ball_control'] = np.nan
                player_dict['interceptions'] = np.nan
                player_dict['penalties'] = np.nan
            
            else:
                fifa_stats = fifa_cleaned.loc[fifa_cleaned['full_name'] == similar_name]
                player_dict['overall_rating'] = int(fifa_stats.overall_rating)
                player_dict['dribbling'] = int(fifa_stats.dribbling)
                player_dict['freekick_accuracy'] = int(fifa_stats.freekick_accuracy)
                player_dict['agility'] = int(fifa_stats.agility)
                player_dict['ball_control'] = int(fifa_stats.ball_control)
                player_dict['interceptions'] = int(fifa_stats.interceptions)
                player_dict['penalties'] = int(fifa_stats.penalties)
            
            team_players.append(player_dict)
        
        # Mean imputation
        team_df = pd.DataFrame(team_players)
        team_df = team_df.dropna()
        
        # Find team means
        overall_rating = np.mean(team_df.overall_rating)
        dribbling = np.mean(team_df.dribbling)
        freekick_accuracy = np.mean(team_df.freekick_accuracy)
        agility = np.mean(team_df.agility)
        ball_control = np.mean(team_df.ball_control)
        interceptions = np.mean(team_df.interceptions)
        penalties = np.mean(team_df.penalties)
        
        # Assign means for missing data entries
        for player_dict in team_players:
            if np.isnan(player_dict['overall_rating']):
                player_dict['overall_rating'] = overall_rating
                player_dict['dribbling'] = dribbling
                player_dict['freekick_accuracy'] = freekick_accuracy
                player_dict['agility'] = agility
                player_dict['ball_control'] = ball_control
                player_dict['interceptions'] = interceptions
                player_dict['penalties'] = penalties
            players_list.append(player_dict)
    return players_list

# Find players & player statistics for each world cup
players_10 = squad_parser(page_soup_10, countries_10)
players_14 = squad_parser(page_soup_14, countries_14)
players_18 = squad_parser(page_soup_18, countries_18)
players_22 = squad_parser(page_soup_22, countries_22)

# Save world cup data
# with open("data/players_10.json", "w") as f:
#     json.dump(players_10, f)
    
# with open("data/players_14.json", "w") as f:
#     json.dump(players_14, f)
    
# with open("data/players_18.json", "w") as f:
#     json.dump(players_18, f)
    
# with open("data/players_22.json", "w") as f:
#     json.dump(players_22, f)

In [41]:
# Reformat data into dataframes
# players_10_df = pd.DataFrame(players_10)
# players_14_df = pd.DataFrame(players_14)
# players_18_df = pd.DataFrame(players_18)
# players_22_df = pd.DataFrame(players_22)

# Save data as CSVs
# players_10_df.to_csv('data/players_2010_world_cup.csv')
# players_14_df.to_csv('data/players_2014_world_cup.csv')
# players_18_df.to_csv('data/players_2018_world_cup.csv')
# players_22_df.to_csv('data/players_2022_world_cup.csv')

# players_22_df.head(10)

In [42]:
# Read datasets 
players_2022 = pd.read_csv('players_2022_world_cup.csv')
players_2018 = pd.read_csv('players_2018_world_cup.csv')
players_2014 = pd.read_csv('players_2014_world_cup.csv')
players_2010 = pd.read_csv('players_2010_world_cup.csv')
matches = pd.read_csv('matches/WorldCupMatches.csv')

In [43]:
players_2022.head()

,Unnamed: 0,name,position,country,overall_rating,dribbling,freekick_accuracy,agility,ball_control,interceptions,penalties
0,0,Hernán Ismael Galíndez,GK,Ecuador,53.0,62.0,33.0,56.0,56.0,41.0,50.0
1,1,Felix Eduardo Torres Caicedo,DF,Ecuador,71.0,68.0,38.0,72.0,71.0,69.0,45.0
2,2,Piero Martín Hincapié Reyna,DF,Ecuador,64.0,13.0,13.0,35.0,13.0,29.0,20.0
3,3,Robert Abel Arboleda Escobar,DF,Ecuador,66.0,68.0,45.0,59.0,71.0,37.0,44.0
4,4,José Adoni Cifuentes Charcopa,MF,Ecuador,72.0,74.0,77.0,74.0,75.0,26.0,72.0


In [44]:
# For each country: 
#   1. Select the goalie with the highest score, 
#   2. Select the top three forwards 
#   3. Select the top four mid fielders 
#   4. Seelct the top three defense 

scores_2022 = [] 

def largest(n, l): 
    l.sort()
    return l[-n:]

def remaining(l, n):
    leng = len(l)
    l.sort()
    return (l[:leng-n])

# Function to create scores dataset 
def scores(dataframe): 
    score = [] 
    for country in dataframe['country'].unique(): 
        dict = {}
        rest = [] 
        dict['country'] = country 
        gk = dataframe[(dataframe['country'] == country) & (dataframe['position'] == 'GK')]
        dict['GK'] = np.max(list(gk['overall_rating']))
        rest.extend(remaining(list(gk['overall_rating']), 1))
        fw = dataframe[(dataframe['country'] == country) & (dataframe['position'] == 'FW')]
        dict['FW'] = np.mean(largest(3, list(fw['overall_rating'])))
        rest.extend(remaining(list(fw['overall_rating']), 3))
        mf = dataframe[(dataframe['country'] == country) & (dataframe['position'] == 'MF')]
        dict['MF'] = np.mean(largest(4, list(mf['overall_rating'])))
        rest.extend(remaining(list(mf['overall_rating']), 4))
        df = dataframe[(dataframe['country'] == country) & (dataframe['position'] == 'DF')]
        dict['DF'] = np.mean(largest(3, list(df['overall_rating'])))
        rest.extend(remaining(list(df['overall_rating']), 3))
        dict['backup'] = np.mean(rest)
        dict['score'] = np.mean([dict['GK'], dict['FW'], dict['MF'], dict['DF'], 0.5*dict['backup']])
        score.append(dict)
    return pd.DataFrame(score)

In [45]:
# Function to create interaction terms for all predictors 
def interaction (score_year): 
    data_year = score_year.drop(['country', 'score'], axis=1)
    poly = PolynomialFeatures(2, include_bias=False)
    poly_year = poly.fit_transform(data_year)
    col = poly.get_feature_names_out(data_year.columns)
    poly_year = pd.DataFrame(poly_year, columns = col)
    for i in col:
        if i not in score_year.columns:
            poly_year[i] = poly_year[i]/100
    poly_year.insert(0,'country', score_year['country'])
    return poly_year

In [46]:
# Get the scores of all the teams 
score_2022 = scores(players_2022)
score_2018 = scores(players_2018)
score_2014 = scores(players_2014)
score_2010 = scores(players_2010)

In [47]:
# Use scores dataset to create interaction terms dataset 
interaction_2022 = interaction(score_2022)
interaction_2018 = interaction(score_2018)
interaction_2014 = interaction(score_2014)
interaction_2010 = interaction(score_2010)

In [48]:
matches.head()

,Year,Datetime,Stage,Stadium,City,Home Team Name,Home Team Goals,Away Team Goals,Away Team Name,Win conditions,Attendance,Half-time Home Goals,Half-time Away Goals,Referee,Assistant 1,Assistant 2,RoundID,MatchID,Home Team Initials,Away Team Initials
0,1930.0,13 Jul 1930 - 15:00,Group 1,Pocitos,Montevideo,France,4.0,1.0,Mexico,,4444.0,3.0,0.0,LOMBARDI Domingo (URU),CRISTOPHE Henry (BEL),REGO Gilberto (BRA),201.0,1096.0,FRA,MEX
1,1930.0,13 Jul 1930 - 15:00,Group 4,Parque Central,Montevideo,USA,3.0,0.0,Belgium,,18346.0,2.0,0.0,MACIAS Jose (ARG),MATEUCCI Francisco (URU),WARNKEN Alberto (CHI),201.0,1090.0,USA,BEL
2,1930.0,14 Jul 1930 - 12:45,Group 2,Parque Central,Montevideo,Yugoslavia,2.0,1.0,Brazil,,24059.0,2.0,0.0,TEJADA Anibal (URU),VALLARINO Ricardo (URU),BALWAY Thomas (FRA),201.0,1093.0,YUG,BRA
3,1930.0,14 Jul 1930 - 14:50,Group 3,Pocitos,Montevideo,Romania,3.0,1.0,Peru,,2549.0,1.0,0.0,WARNKEN Alberto (CHI),LANGENUS Jean (BEL),MATEUCCI Francisco (URU),201.0,1098.0,ROU,PER
4,1930.0,15 Jul 1930 - 16:00,Group 1,Parque Central,Montevideo,Argentina,1.0,0.0,France,,23409.0,0.0,0.0,REGO Gilberto (BRA),SAUCEDO Ulises (BOL),RADULESCU Constantin (ROU),201.0,1085.0,ARG,FRA


In [49]:
matches['Year'].unique()

array([1930., 1934., 1938., 1950., 1954., 1958., 1962., 1966., 1970.,
       1974., 1978., 1982., 1986., 1990., 1994., 1998., 2002., 2006.,
       2010., 2014.,   nan])

In [50]:
matches.columns

Index(['Year', 'Datetime', 'Stage', 'Stadium', 'City', 'Home Team Name',
       'Home Team Goals', 'Away Team Goals', 'Away Team Name',
       'Win conditions', 'Attendance', 'Half-time Home Goals',
       'Half-time Away Goals', 'Referee', 'Assistant 1', 'Assistant 2',
       'RoundID', 'MatchID', 'Home Team Initials', 'Away Team Initials'],
      dtype='object')

In [51]:
matches.shape

(4572, 20)

In [52]:
# Data cleaning 
matches['Home Team Name'] = matches['Home Team Name'].replace(['Korea Republic'], 'South Korea')
matches['Away Team Name'] = matches['Away Team Name'].replace(['Korea Republic'], 'South Korea')
matches['Away Team Name'] = matches['Away Team Name'].replace(['USA'], 'United States')
matches['Home Team Name'] = matches['Home Team Name'].replace(['USA'], 'United States')
matches['Away Team Name'] = matches['Away Team Name'].replace(['C�te d\'Ivoire'], 'Ivory Coast')
matches['Home Team Name'] = matches['Home Team Name'].replace(['C�te d\'Ivoire'], 'Ivory Coast')
matches['Away Team Name'] = matches['Away Team Name'].replace(['Korea DPR'], 'North Korea')
matches['Home Team Name'] = matches['Home Team Name'].replace(['Korea DPR'], 'North Korea')
matches['Home Team Name'] = matches['Home Team Name'].replace(['rn">Bosnia and Herzegovina'], 'Bosnia and Herzegovina')
matches['Away Team Name'] = matches['Away Team Name'].replace(['rn">Bosnia and Herzegovina'], 'Bosnia and Herzegovina')
matches['Home Team Name'] = matches['Home Team Name'].replace(['IR Iran'], 'Iran')
matches['Away Team Name'] = matches['Away Team Name'].replace(['IR Iran'], 'Iran')
matches_2010 = matches[(matches['Year'] == 2010)]
matches_2014 = matches[matches['Year'] == 2014]

In [53]:
# Create final design matrix by merging score / interaction dataset with matches dataset 
def dataset(data, score, year): 
    # data is the matches dataset
    # score is the interaction_year dataset 
    # year is year of game
    match = []
    for home, away, h_score, a_score in zip(data['Home Team Name'], data['Away Team Name'], data['Home Team Goals'], data['Away Team Goals']):
        dict = {}
        dict2 = {}
        dict['match'] = f"{home}-{away}"
        dict2['match'] = f"{away}-{home}"
        h = score[score['country'] == home]
        a = score[score['country'] == away]
        col = score.drop('country', axis=1).columns
        for val in col: 
            dict[f'{val} Difference'] = float(h[val]) - float(a[val])
            dict2[f'{val} Difference'] = float(a[val]) - float(h[val])
        dict['Year'] = year
        dict2['Year'] = year
        if h_score > a_score:
            dict['Winner'] = 2
            dict2['Winner'] = 0
        elif h_score < a_score:
            dict['Winner'] = 0
            dict2['Winner'] = 2
        else: 
            dict['Winner'] = 1
            dict2['Winner'] = 1
        match.append(dict)
        match.append(dict2)
    return pd.DataFrame(match)

In [54]:
output1 = dataset(matches_2010, interaction_2010, 2010)
output2 = dataset(matches_2014, interaction_2014, 2014)

In [55]:
matches_2018 = pd.read_csv('FIFAallMatchBoxData.csv')

In [56]:
matches_2018 = matches_2018[matches_2018['year'] == 2018]
matches_2018['hname'] = matches_2018['hname'].replace(['IR Iran'], 'Iran')
matches_2018['aname'] = matches_2018['aname'].replace(['IR Iran'], 'Iran')
matches_2018['hname'].unique()
matches_2018 = matches_2018.rename(columns={'hname': 'Home Team Name', 'aname': 'Away Team Name'})
matches_2018 = matches_2018.rename(columns={'hgoals': 'Home Team Goals', 'agoals': 'Away Team Goals'})

In [57]:
matches_2018

,year,Home Team Name,Away Team Name,Home Team Goals,Away Team Goals,hPossesion,aPossesion,hshotsOnTarget,ashotsOnTarget,hshots,ashots,hyellowCards,ayellowCards,hredCards,aredCards,hfouls,afouls,hsaves,asaves
1411,2018,Russia,Saudi Arabia,5,0,41,59,7,0,13,6,1,1,0,0,22,10,0,2
1412,2018,Egypt,Uruguay,0,1,43,58,3,4,8,14,2,0,0,0,12,6,3,3
1413,2018,Morocco,Iran,0,1,63,37,3,2,13,8,1,3,0,0,22,14,2,2
1414,2018,Portugal,Spain,3,3,38,62,3,5,8,12,1,1,0,0,12,10,2,0
1415,2018,France,Australia,2,1,51,49,5,1,13,4,1,3,0,0,16,19,1,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1470,2018,Russia,Croatia,2,2,36,64,7,3,13,17,1,4,0,0,25,18,1,5
1471,2018,France,Croatia,4,2,39,61,6,3,8,15,2,1,0,0,14,13,1,3
1472,2018,France,Belgium,1,0,40,60,5,3,19,9,2,3,0,0,6,16,3,4
1473,2018,Croatia,England,2,1,54,46,7,1,22,11,2,1,0,0,23,14,0,5


In [58]:
output3 = dataset(matches_2018, interaction_2018, 2018)

In [59]:
d = [output1, output2, output3]
result = pd.concat(d)

In [60]:
# result.to_csv('df_interaction.csv')

In [61]:
main_2010 = dataset(matches_2010, score_2010, 2010)
main_2014 = dataset(matches_2014, score_2014, 2014)
main_2018 = dataset(matches_2018, score_2018, 2018)

In [62]:
# Data set without interaction terms 
ds = [main_2010, main_2014, main_2018]
result2 = pd.concat(ds)
result2 = result2.drop('score Difference', axis=1)

In [63]:
# result2.to_csv('df_main.csv')

In [64]:
# Function to get all features as their own column
def individual(data, score, year): 
    match = []
    for home, away, h_score, a_score in zip(data['Home Team Name'], data['Away Team Name'], data['Home Team Goals'], data['Away Team Goals']):
        dict = {}
        dict['match'] = f"{home}-{away}"
        h = score[score['country'] == home]
        a = score[score['country'] == away]
        col = score.drop('country', axis=1).columns
        for val in col: 
            dict[f'{val} Home'] = float(h[val]) 
            dict[f'{val} Away'] = float(a[val])
        dict['Year'] = year
        if h_score > a_score:
            dict['Winner'] = 2
        elif h_score < a_score:
            dict['Winner'] = 0
        else: 
            dict['Winner'] = 1
        match.append(dict)
    return pd.DataFrame(match)

In [65]:
# Extension of the individual function to include the reverse 
def individual_reverse(data, score, year): 
    match = []
    for home, away, h_score, a_score in zip(data['Home Team Name'], data['Away Team Name'], data['Home Team Goals'], data['Away Team Goals']):
        dict = {}
        dict2 = {}
        dict['match'] = f"{home}-{away}"
        dict2['match'] = f"{away}-{home}"
        h = score[score['country'] == home]
        a = score[score['country'] == away]
        col = score.drop(['country', 'score'], axis=1).columns
        for val in col: 
            dict[f'{val} Home'] = float(h[val]) 
            dict[f'{val} Away'] = float(a[val])
            dict2[f'{val} Home'] = float(a[val])
            dict2[f'{val} Away'] = float(h[val])
        dict['Year'] = year
        dict2['Year'] = year
        if h_score > a_score:
            dict['Winner'] = 2
            dict2['Winner'] = 0
        elif h_score < a_score:
            dict['Winner'] = 0
            dict2['Winner'] = 2
        else: 
            dict['Winner'] = 1
            dict2['Winner'] = 1
        match.append(dict)
        match.append(dict2)
    return pd.DataFrame(match)

In [66]:
# Create individual dataset 
individual_2010 = individual(matches_2010, score_2010, 2010)
individual_2014 = individual(matches_2014, score_2014, 2014)
individual_2018 = individual(matches_2018, score_2018, 2018)
d3 = [individual_2010, individual_2014, individual_2018]
result3 = pd.concat(d3)
result3 = result3.drop(['score Home', 'score Away'], axis=1)
result3.to_csv('df_individual.csv')

In [67]:
# Create individual + reverse dataset 
individual_r2010 = individual_reverse(matches_2010, score_2010, 2010)
individual_r2014 = individual_reverse(matches_2014, score_2014, 2014)
individual_r2018 = individual_reverse(matches_2018, score_2018, 2018)
d4 = [individual_r2010, individual_r2014, individual_r2018]
result4 = pd.concat(d4)
# result4.to_csv('df_individual_reverse.csv')

In [68]:
# Create 2022 dataset 
group_matches_2022 = pd.read_csv('group_stage_22.csv')
matches_2022 = group_matches_2022.iloc[:48, [0,1]]
matches_2022['Home Team'] = matches_2022['Home Team'].replace(['IR Iran'], 'Iran')
matches_2022['Away Team'] = matches_2022['Away Team'].replace(['IR Iran'], 'Iran')
matches_2022['Home Team'] = matches_2022['Home Team'].replace(['Korea Republic'], 'South Korea')
matches_2022['Away Team'] = matches_2022['Away Team'].replace(['Korea Republic'], 'South Korea')
matches_2022['Home Team'] = matches_2022['Home Team'].replace(['USA'], 'United States')
matches_2022['Away Team'] = matches_2022['Away Team'].replace(['USA'], 'United States')

In [69]:
# Individual reverse function for 2022 
def individual_reverse_2022(data, score, year): 
    match = []
    for home, away in zip(data['Home Team'], data['Away Team']):
        dict = {}
        dict2 = {}
        dict['match'] = f"{home}-{away}"
        dict2['match'] = f"{away}-{home}"
        h = score[score['country'] == home]
        a = score[score['country'] == away]
        col = score.drop(['country', 'score'], axis=1).columns
        for val in col: 
            dict[f'{val} Home'] = float(h[val]) 
            dict[f'{val} Away'] = float(a[val])
            dict2[f'{val} Home'] = float(a[val])
            dict2[f'{val} Away'] = float(h[val])
        dict['Year'] = year
        dict2['Year'] = year
        match.append(dict)
        match.append(dict2)
    return pd.DataFrame(match)

In [70]:
df_2022 = individual_reverse_2022(matches_2022, score_2022, 2022)

In [71]:
# df_2022.to_csv('df_2022.csv')

In [72]:
# score_2022.to_csv('score_2022.csv')